# Finite elements

This notebook explores setting up finite elements and messing with them

## Convecting incompressible fluid flow

In [1]:
from dataclasses import dataclass

import numpy as np
import triangle as tr

import matplotlib.pyplot as plt
import matplotlib.tri as mtri

In [2]:
from scipy import sparse, linalg as lin
import itertools

In [3]:
from matplotlib.colors import Normalize as mpl_normalize


In [4]:
def plot_triangulation(ax, B, **kwargs):
    ax.triplot(
        B["vertices"][:, 0],
        B["vertices"][:, 1],
        B["triangles"],
        **kwargs
    )
    ax.scatter(B["vertices"][:, 0],
               B["vertices"][:, 1],
              c="k",
              s=1)
    ax.set_aspect("equal")
    return ax

In [5]:
def plot_function(ax, function, triangulation, **kwargs):
    ax.tricontourf(
        triangulation["vertices"][:, 0],
        triangulation["vertices"][:, 1],
        triangulation["triangles"],
        function,
        **kwargs
    )
    ax.set_aspect("equal")
    return

In [6]:
# https://matplotlib.org/stable/gallery/mplot3d/trisurf3d_2.html

def plot_graph(function, triangulation, **kwargs):
    fig = plt.figure(figsize=plt.figaspect(0.4))

    ax = fig.add_subplot(1, 2, 1, projection="3d")

    ax.plot_trisurf(
        mtri.Triangulation(
            x=triangulation["vertices"][:,0],
            y=triangulation["vertices"][:,1],
            triangles=triangulation["triangles"]
        ),
        function,
        **kwargs
    )
    plt.show()

In [7]:
def rot(v):
    """90 degree cw rotation of vector
    """
    return np.array([[0, 1], [-1, 0]]).dot(v)


In [ ]:
def operators( tri ):
    """Assemble Laplacian matrix L associated to geometry tri

    L_{ij} = \sum_{faces adj p_i, p_j} -\langle p_j - p_k, p_i - p_k\rangle / 4A^2
    
    Accomplish by summing over faces
    """

    vertices = tri["vertices"] # shape = (N, 2)
    faces = tri["triangles"] # shape = (N, 3) and each col is an array of ints
    
    num_vert = vertices.shape[0]
    num_faces = faces.shape[0]

    l2_ind_I = []
    l2_ind_J = []
    l2_values = []
    
    lapl_ind_I = []
    lapl_ind_J = []
    lapl_values = []
    
    grad_ind_I = []
    grad_ind_J = []
    grad_ind_K = []
    grad_values = []

    div_ind_I = []
    div_ind_J = []
    div_ind_K = []
    div_values = []
#     l2_inner_product = np.zeros( (num_vert, num_vert) )
#     laplacian = np.zeros( (num_vert, num_vert) )
    
#     grad = np.zeros( (2, num_faces, num_vert) )
#     div = np.zeros( (num_vert, num_faces, 2) )    
    
    for face_ind in range( num_faces ):
        
        face = faces[ face_ind, : ]
        
        ind = {
            face[0]: 0,
            face[1]: 1,
            face[2]: 2
        }
        
        # vertices of the face
        v0 = vertices[ face[0], : ]
        v1 = vertices[ face[1], : ]
        v2 = vertices[ face[2], : ]
        
        # area of the face
        area = np.abs( np.cross( v2 - v0, v1 - v0 ) )/ 2.0

        # barycentric embedding
        A = np.array([
            [ v0[0], v1[0], v2[0] ],
            [ v0[1], v1[1], v2[1] ],
            [ 1.,    1.,    1.    ]
        ])
        
        # invert to find the gradients of the element functions
        B = lin.inv( A )
        # the gradient of jth element function phi_j is 
        # B[ j, :2 ]
        
        for i, j in itertools.product( face, face ):
            
            loc_i = ind[i]
            loc_j = ind[j]
            
            lapl_ind.append(
            
            laplacian[i, j] += area * B[ loc_i, :2 ].dot( B[ loc_j, :2 ] )
                                                                 
            if i == j:
                l2_inner_product[i, j] += area / 6
            else:
                l2_inner_product[i, j] += area / 12
              
        for k in range(len(face)):
            idx = face[k]
            grad[ :, face_ind, idx ] += B[ k, :2 ]
            div[ idx, face_ind, : ] += - B[ k, :2 ].T / lin.norm( B[k, :2] )
    return {
        "inner_product": l2_inner_product,
        "gradient": grad,
        "divergence": div,
        "laplacian": laplacian
    }

In [ ]:
example = tr.triangulate({
    "vertices": np.array([[0, 0], [1, 0], [1, 1], [0, 1]])
}, "qa2")

In [ ]:
example["vertices"].shape

In [ ]:
example["vertices"][ 18, : ]

In [ ]:
example["triangles"].shape

In [ ]:
example["triangles"][ 1127, : ]

In [ ]:
ops = operators(example)

M = ops["inner_product"]
grad = ops["gradient"]
div = ops["divergence"]
L = ops["laplacian"]

In [ ]:
def height(x, y):
    height_m = 10\
        + ( 2500 / (1 + np.exp( -0.2 * (y - 25) ) )\
            + np.random.normal( 0, 50 )\
          ) * (1 - 0.2*np.sin(x / 5)**2)
    height_km = height_m / 1000.
    return height_km

In [ ]:
h = height( example["vertices"][:,0], example["vertices"][:,1] )

In [ ]:
fig, ax = plt.subplots()
plot_function( ax, h, example, cmap="terrain")
plot_triangulation( ax, example, c="k", lw=1 )
fig.set_size_inches(12, 12)
plt.show()

In [ ]:
plot_graph(h, example, cmap="terrain")

In [ ]:
laplacian_of_height = np.dot(L, h)
plot_graph(laplacian_of_height, example, cmap="copper")

In [ ]:
gradient_of_height = np.dot(grad, h)

In [ ]:
# calculate triangular face centroids
face_centers = np.zeros( (2, example["triangles"].shape[0]) )

for f_ind in range(len(example["triangles"])):
    face = example["triangles"][ f_ind, : ]
    v0 = example["vertices"][ face[0], : ]
    v1 = example["vertices"][ face[1], : ]
    v2 = example["vertices"][ face[2], : ]
    
    center = np.average( np.array( [v0, v1, v2] ), axis=0 )

    face_centers[ :, f_ind ] += center

In [ ]:
x = face_centers[0, :]
y = face_centers[1, :]
u = gradient_of_height[0, :]
v = gradient_of_height[1, :]

fig, ax = plt.subplots()
plot_function( ax, h, example, cmap="terrain")
# plot_triangulation( ax, example, c="k", lw=1 )
ax.quiver( x, y, u, v )
fig.set_size_inches(12, 12)
plt.show()

In [ ]:
gradient_of_height.shape

In [ ]:
div.shape

In [ ]:
divgrad_of_height = div[ :, :, 0].dot( gradient_of_height[0, :] ) + div[ :, :, 1 ].dot( gradient_of_height[1, :])

In [ ]:
fig, (ax0, ax1) = plt.subplots( ncols=2 )

plot_function( ax0, laplacian_of_height, example, cmap="terrain")
plot_triangulation( ax0, example, c="k", lw=1 )

plot_function( ax1, divgrad_of_height, example, cmap="terrain")
plot_triangulation( ax1, example, c="k", lw=1 )

fig.set_size_inches(12, 12)
plt.show()

# Testing on the Euler equation

https://en.wikipedia.org/wiki/Euler_equations_(fluid_dynamics)#Incompressible_Euler_equations_with_constant_and_uniform_density

## State

velocity


In [ ]:
pipe = tr.triangulate({
    # "vertices": np.array([[0, 0], [2, 0], [2, 1], [0, 1]])   # square
    "vertices": np.array( [ [np.cos(th), np.sin(th)] for th in np.arange(0, 2*np.pi, 0.1) ] )
}, "qa0.0003")

num_vert = pipe["vertices"].shape[0]
num_faces = pipe["triangles"].shape[0]

# calculate triangular face centroids
face_centers = np.zeros( (2, pipe["triangles"].shape[0]) )

for f_ind in range(len(pipe["triangles"])):
    face = pipe["triangles"][ f_ind, : ]
    v0 = pipe["vertices"][ face[0], : ]
    v1 = pipe["vertices"][ face[1], : ]
    v2 = pipe["vertices"][ face[2], : ]
    
    center = np.average( np.array( [v0, v1, v2] ), axis=0 )

    face_centers[ :, f_ind ] += center

ops = operators( pipe )

M = ops["inner_product"]
grad = ops["gradient"]
div = ops["divergence"]
L = ops["laplacian"]

In [ ]:
fig, ax = plt.subplots()
plot_triangulation( ax, pipe, c="k", lw=1 )
fig.set_size_inches(6, 6)
plt.show()

In [ ]:
def vertex_mean_vf( u ):
    """Calculate the mean at each vertex of a given vector field
    this is done by averaging the values over each face weighted by the angle of the vertex in that face
    """    
    umean = np.zeros( (2, num_vert) )
    for j in range(num_faces):
        face = pipe["triangles"][ j, : ]
        val = u[ :, j ]
        v0 = pipe["vertices"][ face[0], : ]
        v1 = pipe["vertices"][ face[1], : ]
        v2 = pipe["vertices"][ face[2], : ]
        
        theta0 = np.arccos( np.dot( v2 - v0, v1 - v0 ) / ( lin.norm( v2 - v0 )*lin.norm( v1 - v0 ) ) )
        theta1 = np.arccos( np.dot( v2 - v0, v1 - v0 ) / ( lin.norm( v2 - v0 )*lin.norm( v1 - v0 ) ) )
        theta2 = np.arccos( np.dot( v2 - v0, v1 - v0 ) / ( lin.norm( v2 - v0 )*lin.norm( v1 - v0 ) ) )
        
        umean[:, face[0]] = val * theta0 / (2*np.pi)
        umean[:, face[1]] = val * theta1 / (2*np.pi)
        umean[:, face[2]] = val * theta2 / (2*np.pi)
        
    return umean

def vertex_mean_func( u ):
    """Calculate the mean at each vertex of a given vector field
    this is done by averaging the values over each face weighted by the angle of the vertex in that face
    """    
    umean = np.zeros( num_vert )
    for j in range(num_faces):
        face = pipe["triangles"][ j, : ]
        val = u[ j ]
        v0 = pipe["vertices"][ face[0], : ]
        v1 = pipe["vertices"][ face[1], : ]
        v2 = pipe["vertices"][ face[2], : ]
        
        theta0 = np.arccos( np.dot( v2 - v0, v1 - v0 ) / ( lin.norm( v2 - v0 )*lin.norm( v1 - v0 ) ) )
        theta1 = np.arccos( np.dot( v2 - v0, v1 - v0 ) / ( lin.norm( v2 - v0 )*lin.norm( v1 - v0 ) ) )
        theta2 = np.arccos( np.dot( v2 - v0, v1 - v0 ) / ( lin.norm( v2 - v0 )*lin.norm( v1 - v0 ) ) )
        
        umean[face[0]] = val * theta0 / (2*np.pi)
        umean[face[1]] = val * theta1 / (2*np.pi)
        umean[face[2]] = val * theta2 / (2*np.pi)
        
    return umean

def face_mean( u ):
    """Calculate mean of function on each face by averaging values over each face
    """
    pass

In [ ]:
def take_divergence( u ):
    """Take divergence of vector field u
    """
    return np.dot( div[ :, :, 0], u[0, :] ) + np.dot( div[ :, :, 1], u[1, :] )

In [ ]:
def state_update( dt, velocity, pressure, gravity ):
    
    vel_at_vert = vertex_mean_vf(velocity)
    
    vav_x = vel_at_vert[ 0, : ]
    vav_y = vel_at_vert[ 1, : ]
    
    gradvav_x = np.dot( grad, vav_x )
    gradvav_y = np.dot( grad, vav_y )
    
    cov_deriv_velocity = np.array( [ np.sum(velocity[ 0, : ] * gradvav_x, axis=0),
                                     np.sum(velocity[ 1, : ] * gradvav_y, axis=0) ] )
    
    pressure = lin.solve( take_divergence( grad ), take_divergence(cov_deriv_velocity) )
    
    dvelocity = - np.dot( grad, pressure ) + gravity - cov_deriv_velocity
    
    new_velocity = velocity + dt * dvelocity
    
    return {
        "velocity": new_velocity,
        "pressure": pressure,
    }

In [ ]:
import imageio

In [ ]:
# TODO: animate via matplotlib
# https://matplotlib.org/stable/api/animation_api.html
# double pendulum example: 
#     https://matplotlib.org/stable/gallery/animation/double_pendulum.html

In [ ]:
# Alternatively:
# https://towardsdatascience.com/basics-of-gifs-with-pythons-matplotlib-54dd544b6f30

In [ ]:
velocity = np.zeros( (2, num_faces) )
# velocity = np.array([ -face_centers[1, :], face_centers[0, :] ])
gravity = np.zeros( (2, num_faces) )
gravity[ 1, : ] = -9.8 * np.ones( num_faces )
pressure = np.ones( num_vert ) # this is specific thermodynamic work which is the same as mechanical pressure

x = face_centers[0, :]
y = face_centers[1, :]

In [ ]:
!mkdir tmp

In [ ]:
filenames = []
for t in range(100):
    try:
        next_state = state_update( 0.005,  velocity, pressure, gravity )
        velocity = next_state["velocity"]
        pressure = next_state["pressure"]
    except Exception as err:
        print("Oh no!")
        print(err)
        print("Breaking loop now! x_x")
        break

    u = velocity[0, :]
    v = velocity[1, :]

    div_velocity = np.dot( div[ :, :, 0], velocity[0, :] ) + np.dot( div[ :, :, 1], velocity[1, :] )

    fig, ax = plt.subplots()

    # plot_triangulation( ax, pipe, c="k", lw=1 )
    ax.quiver( x, y, u, v )
    ax.set_aspect("equal")
    fig.set_size_inches(6, 6)

    # https://stackoverflow.com/questions/63754046/when-adding-colorbar-for-a-matplotlib-tricontourf-typeerror-you-must-first-se
    # plot_function( ax, pressure, pipe, cmap="terrain")
    # norm = mpl_normalize(vmin=np.min(div_velocity), vmax=np.max(div_velocity))
    # fig.colorbar( plt.cm.ScalarMappable(cmap="terrain", norm=norm ) )

    filename = f"tmp/{str(t).zfill(2)}.png"
    filenames.append(filename)
    fig.savefig( filename )
    if t % 10 == 0:
        print(f"Saved {filename}")
    
    plt.close()
    # plt.show()

In [ ]:
with imageio.get_writer("mesh_fineness_003.gif", mode="I") as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

In [ ]:
! rm -rf tmp